# SparK MLib - Credit Card Fraud Classification

Data Source: Kaggle.com

In [ ]:
#Let's mount Google Drive So We can Retrieve the Data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#There is One File of Creit Card Data. Source  Kaggle.com
!ls "/content/gdrive/My Drive/CC Data/"

creditcardfraud.zip


In [ ]:
#Unzip the files to reveal creditcard.csv
#!rm -r *
!unzip -qq "/content/gdrive/My Drive/CC Data/creditcardfraud.zip"

In [ ]:
!ls

creditcard.csv	gdrive	sample_data


# **Install and Load Up Spark**

In [ ]:
#Install Latest Version of Spark As of Current Data. 2.4.3

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.format("csv").option("inferSchema", True).option("header", True).load('creditcard.csv').withColumnRenamed('Class', 'label')
df.toPandas().head(5).style.hide_index()

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,label
0,-1.35981,-0.0727812,2.53635,1.37816,-0.338321,0.462388,0.239599,0.0986979,0.363787,0.0907942,-0.5516,-0.617801,-0.99139,-0.311169,1.46818,-0.470401,0.207971,0.0257906,0.403993,0.251412,-0.0183068,0.277838,-0.110474,0.0669281,0.128539,-0.189115,0.133558,-0.0210531,149.62,0
0,1.19186,0.266151,0.16648,0.448154,0.0600176,-0.0823608,-0.078803,0.0851017,-0.255425,-0.166974,1.61273,1.06524,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.0690831,-0.225775,-0.638672,0.101288,-0.339846,0.16717,0.125895,-0.0089831,0.0147242,2.69,0
1,-1.35835,-1.34016,1.77321,0.37978,-0.503198,1.8005,0.791461,0.247676,-1.51465,0.207643,0.624501,0.0660837,0.717293,-0.165946,2.34586,-2.89008,1.10997,-0.121359,-2.26186,0.52498,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.0553528,-0.0597518,378.66,0
1,-0.966272,-0.185226,1.79299,-0.863291,-0.0103089,1.2472,0.237609,0.377436,-1.38702,-0.0549519,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.05965,-0.684093,1.96578,-1.23262,-0.208038,-0.1083,0.0052736,-0.190321,-1.17558,0.647376,-0.221929,0.0627228,0.0614576,123.5,0
2,-1.15823,0.877737,1.54872,0.403034,-0.407193,0.0959215,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.34585,-1.11967,0.175121,-0.451449,-0.237033,-0.0381948,0.803487,0.408542,-0.0094307,0.798278,-0.137458,0.141267,-0.20601,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df.printSchema()

root
 |-- Time: decimal(10,0) (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double

####ANALYSIS

Examining the table above. This data is a collection of PCA features (to protect privacy) along with an amount total and a class total.

In [ ]:
# total amount of records is 284807
print('Total amount of records is', df.count())

Total amount of records is 284807


In [ ]:
df.describe().toPandas().style.hide_index().set_properties(**{'background-color': 'lightgrey', 'color': 'Black','border-color': 'white', "text-align" : "right"})

summary,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,label
count,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807
mean,94813.8596,5.1093952144318264E-17,2.554697607215913E-16,-8.634877912389787E-15,3.5254826979579603E-15,2.0437580857727305E-15,2.088465293899009E-15,-2.2864543584582423E-15,-3.5765766501022783E-16,-1.3412162437883544E-15,1.9000563453668355E-15,2.05653157380881E-15,-2.672852371549649E-15,7.887628862279132E-16,1.4976914722303291E-15,2.707979463648868E-15,1.444202491079246E-15,-3.1933720090198914E-17,3.640444090282676E-16,9.468348006743977E-16,1.820222045141338E-16,2.554697607215913E-16,8.941441625255696E-16,7.664092821647739E-16,4.515428020754126E-15,1.5455920523656276E-15,1.737194372906821E-15,-3.5456408587648983E-16,-1.311278381203793E-16,88.3496192509508,0.001727485630620034
stddev,47488.145954566906,1.9586958038574882,1.651308579476995,1.5162550051777774,1.4158685749409223,1.3802467340314277,1.3322710897575698,1.2370935981826607,1.1943529026691984,1.0986320892243164,1.0888497654025246,1.0207130277115635,0.9992013895301407,0.9952742301251535,0.958595611257067,0.9153160116104393,0.8762528873883741,0.8493370636743898,0.8381762095288434,0.8140405007685821,0.7709250248871147,0.7345240143713111,0.7257015604409144,0.6244602955949919,0.605647067827159,0.5212780705409377,0.48222701326105777,0.4036324949650308,0.33008326416025036,250.12010924018736,0.04152718963546528
min,0,-56.407509631329,-72.7157275629303,-48.3255893623954,-5.68317119816995,-113.743306711146,-26.1605059358433,-43.5572415712451,-73.2167184552674,-13.4340663182301,-24.5882624372475,-4.79747346479757,-18.6837146333443,-5.79188120632084,-19.2143254902614,-4.49894467676621,-14.1298545174931,-25.1627993693248,-9.49874592104677,-7.21352743017759,-54.497720494566,-34.8303821448146,-10.933143697655,-44.8077352037913,-2.83662691870341,-10.2953970749851,-2.60455055280817,-22.5656793207827,-15.4300839055349,0.0,0
max,172792,2.45492999121121,22.0577289904909,9.38255843282114,16.8753440335975,34.8016658766686,73.3016255459646,120.589493945238,20.0072083651213,15.5949946071278,23.7451361206545,12.0189131816199,7.8483920756446,7.12688295859376,10.5267660517847,8.87774159774277,17.3151115176278,9.25352625047285,5.04106918541184,5.59197142733558,39.4209042482199,27.2028391573154,10.5030900899454,22.5284116897749,4.58454913689817,7.51958867870916,3.5173456116238,31.6121981061363,33.8478078188831,25691.16,1


In [ ]:
#check missing values for each column
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(column), column)).alias('MV') for column in df.columns]).toPandas()

,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV,MV
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Lets Check the distribution of the class feature. Note  0 = No Fraud, while 1 = Fraud
df.groupby('label').count().toPandas().style.hide_index().set_properties(**{'background-color': 'lightgrey', 'color': 'Black','border-color': 'white', "text-align" : "right"})

label,count
1,492
0,284315


###Unbalanced Class Problem
As you can see above, the fraud class (1) is only 492 records out to the 284315. This is a unbalanced class situation.

In [ ]:
#Split the data train vs test

weights = [.8, .2]  #80% Train, 20% Test given the unbalanced nature of the data
seed = 42
dfTrain, dfTest = df.randomSplit(weights, seed)
dfTrain.cache(), dfTest.cache()  #IMPORTANT TO CACHE!!!!
print('The Train count is', dfTrain.count(), 'and the Test count is', dfTest.count())

The Train count is 228135 and the Test count is 56672


# Running A Model On the Unbalanced Data

In [ ]:
#Logistic Regression Model
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression().setLabelCol("label")
paramMap = {lr.maxIter: 10, lr.regParam: .1, lr.elasticNetParam: 0.01}

In [ ]:
#All our features are numberical, we can use VectorAssembler right away to create a feature vector that Spark needs

from pyspark.ml.feature import VectorAssembler

#assemble our features
IntFeat = df.columns[1:31]
assembler = VectorAssembler(inputCols= IntFeat, outputCol = "features")

In [ ]:
#Define a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler] + [lr])
model_Train = pipeline.fit(dfTrain, paramMap)

In [ ]:
#run the model on the Test Data (holdout)

output = model_Train.transform(dfTest).select("features","label", "prediction", "rawPrediction", "probability")
prediction = output.select("label", "prediction", "rawPrediction", )

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
metric = evaluator.evaluate(prediction)
print("Accuracy = %s" % metric)

Accuracy = 0.9987648221343873


#### ANALYSIS

Keep in mind, given the unbalanced class, a high accuracy means the model is good at predicting the non fraud class, but what about the fraud class? Let's run some more statistics and a confusion matrix.

In [ ]:
#Metrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics
predRDD = prediction.select('label', 'prediction').withColumn("label", prediction.label.cast('float')).rdd
metrics = BinaryClassificationMetrics(predRDD)

print("Summary Stats:")
print("Area Under Precision Recall = %s" %  metrics.areaUnderPR)
print("Area Under ROC = %s" % metrics.areaUnderROC)

Summary Stats:
Area Under Precision Recall = 0.34579439252336447
Area Under ROC = 0.9993820075924781


In [ ]:
#The above scores for accuracy and Area Under ROC look good, however, when we look at Area Under PR, the results are quite poor.
#This is not a suprise when your classes are heavily unbalanced. Looking at the actual Class vs predected class.
#This model only classifies only 30% of true fraud cases. We can do better.

prediction.crosstab("label", 'prediction').\
toPandas().style.hide_index().set_properties(**{'background-color': 'lightgrey', 'color': 'Black','border-color': 'white', "text-align" : "right"})

label_prediction,0.0,1.0
1,70,37
0,56565,0


#### ANALYSIS
As you can see above, auPR is quite low, while auROC is quite high. This is not unusual when the classes are unbalanced. Taking a look at the confusion matrix, only 37 out of 107 of fraud cases were classified correctly. Not exactly great performance. The model did correctly classify all the non-fraud cases. Given that the point of this exercise is to detect fraud cases, that is not very helpful.

# Running A Model On a More Balanced Dataset

###What to do about Unbalanced Class problem?
In general, there are a number of things you can do to help aleviate the problem of unbalanced classes such as trying different algorithms, collecting more data, sub-sampling the majority class, over-sampling the minority class, etc... In this example, we are going to over-weight the loss function towards the minority class, known fraud cases. This should provide a more robust model. The function below was inspired by the following post...

https://stackoverflow.com/questions/33372838/dealing-with-unbalanced-datasets-in-spark-mllib

In [ ]:
#Lets increase the balance among classes

#from pyspark.sql.functions import udf

def balanceDataset(DataFrame, String = "label"):
  datasetSize= DataFrame.count()                                       
  positives = DataFrame.filter(dfTrain.label == 1).count()
  balancingRatio = positives/datasetSize
  weighteddfTrain = DataFrame.withColumn("classWeightCol", (when(col("label") == 1, 1 - balancingRatio).otherwise(balancingRatio)))
  return weighteddfTrain
  
#spark.udf.register("BalanceDataset", balanceDataset)  

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

weightedDatasetdf= balanceDataset(dfTrain)

In [ ]:
#Logistic Regression Model
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression().setWeightCol('classWeightCol').setLabelCol("label")
#paramMap = {lr.maxIter: 10, lr.regParam: .1, lr.elasticNetParam: 0.01}
paramMap = {lr.maxIter: 5, lr.regParam: .01, lr.elasticNetParam: 0.1}

In [ ]:
#All our features are numberical, we can use VectorAssembler right away to create a feature vector that Spark needs

from pyspark.ml.feature import VectorAssembler

#assemble our features
IntFeat = weightedDatasetdf.columns[1:30]
assembler = VectorAssembler(inputCols= IntFeat, outputCol = "features")

In [ ]:
#Define a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler] + [lr])
model_Train = pipeline.fit(weightedDatasetdf, paramMap)

In [ ]:
#run the model on the Test Data (holdout)
weightedDatasetdftest= balanceDataset(dfTest)

output = model_Train.transform(weightedDatasetdftest).select("features","label", "prediction", "rawPrediction", "probability")
prediction = output.select("label", "prediction")

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
metric = evaluator.evaluate(prediction)
print("Accuracy = %s" % metric)

Accuracy = 0.9945475719932242


In [ ]:
#Metrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics
predRDD = prediction.select('label', 'prediction').withColumn("label", prediction.label.cast('float')).rdd
metrics = BinaryClassificationMetrics(predRDD)

print("Summary Stats:")
print("Area Under Precision Recall = %s" %  metrics.areaUnderPR)
print("Area Under ROC = %s" % metrics.areaUnderROC)

Summary Stats:
Area Under Precision Recall = 0.5413467948775839
Area Under ROC = 0.6197209915546122


In [ ]:
prediction.crosstab("label", 'prediction').\
toPandas().style.hide_index().set_properties(**{'background-color': 'lightgrey', 'color': 'Black','border-color': 'white', "text-align" : "right"})

label_prediction,0.0,1.0
1,14,93
0,56270,295


####ANALYSIS

Over-weighting the loss function to the fraud cases in the logisitc regression model significantly improves our accuracy with respect to the fraud class. AuPR jumps to .61 from .53. All good results. Can we further improve the model by using cross validation and a grid search for hyperparamters?

# Adding Cross-Validation and Grid Search for Hyperparameters

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from  pyspark.ml.evaluation import BinaryClassificationEvaluator

#assemble our features

cvevaluator = BinaryClassificationEvaluator(metricName = 'areaUnderPR')  #can select metric to evaluate
cvparamGrid = ParamGridBuilder().addGrid(lr.maxIter, [1]).addGrid(lr.regParam, [.001, .005, .01, .09])\
.addGrid(lr.elasticNetParam, [.0001, .01, .02, .03]).build()

numFolds=2

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=cvparamGrid,
    evaluator=cvevaluator,
    numFolds=numFolds)

cvModel = crossval.fit(weightedDatasetdf)

#cvPrediction = cvModel.transform(weightedDatasetdftest).select("label", "prediction")
cvPrediction = cvModel.transform(weightedDatasetdftest).select("label", "prediction")

#Note: Automatically selects best model, if you would like to see bestmodel then:
bestModel = cvModel.bestModel

In [ ]:
cvPrediction = output.select("label", "prediction")

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
metric = evaluator.evaluate(cvPrediction)
print("Accuracy = %s" % metric)

Accuracy = 0.9945475719932242


In [ ]:
#Metrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics
predRDD = cvPrediction.select('label', 'prediction').withColumn("label", cvPrediction.label.cast('float')).rdd
metrics = BinaryClassificationMetrics(predRDD)

print("Summary Stats:")
print("Area Under Presion Recall = %s" %  metrics.areaUnderPR)
print("Area Under ROC = %s" % metrics.areaUnderROC)

Summary Stats:
Area Under Presion Recall = 0.5413467948775839
Area Under ROC = 0.6197209915546122


In [ ]:
cvPrediction.crosstab("label", 'prediction').\
toPandas().style.hide_index().set_properties(**{'background-color': 'lightgrey', 'color': 'Black','border-color': 'white', "text-align" : "right"})

label_prediction,0.0,1.0
1,14,93
0,56270,295


In [ ]:
#Look at the parameters for the best model

bestParam = bestModel.stages[1].extractParamMap()
bestParam

{Param(parent='LogisticRegression_63cbaba1add4', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LogisticRegression_63cbaba1add4', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.03,
 Param(parent='LogisticRegression_63cbaba1add4', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto',
 Param(parent='LogisticRegression_63cbaba1add4', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LogisticRegression_63cbaba1add4', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LogisticRegression_63cbaba1add4', name='labelCol', doc='label column name'): 'label',
 Param(parent='LogisticRegression_63cbaba1add4', name='maxIter', doc='maximum number of iterations (>= 0)'):

#### ANALYSIS

Using cross-validation and grid-search, performance has improved slightly on the fraud cases at the cost of performance on the non-fraud cases. In scenarios like this, a business decision needs to be made whether the improvement in capturing more fraud cases is worth incorrectly classifying non-fraud cases.